In [1]:
from ldm import util
from omegaconf import OmegaConf
import torch
from ldm.models.autoencoder import VQModel
import numpy as np

In [2]:
ckpt = torch.load("/home/michel/workspace/gen_modeling/implementation/auto-encoders/model.ckpt")

/tmp/ipykernel_7553/4229985802.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/home/michel/workspace/gen_modeling/implementation/auto-encoders/model.

In [3]:
sd = ckpt['state_dict']
for key in list(sd.keys()):
    if key.startswith("loss."):
        sd.pop(key)

In [4]:
config = OmegaConf.load("models/first_stage_models/vq-f4/config.yaml")

In [5]:
autoencoder = util.instantiate_from_config(config['model'])

making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [6]:
device = torch.device("cuda")
autoencoder.load_state_dict(sd)
autoencoder.eval().to(device)

VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): 

In [7]:
print(f"Parameters: {sum(p.numel() for p in autoencoder.parameters()):,}")

Parameters: 55,322,782


In [8]:
from PIL import Image
from torchvision.transforms import v2

In [9]:
img = Image.open("/home/michel/workspace/gen_modeling/implementation/misc/0810.png")
# convert to tensor
img_tensor = torch.tensor(np.array(img).transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0) / 255.0
img_tensor = v2.Resize((512, 512))(img_tensor).to(device)

In [10]:
with torch.no_grad():
    z = autoencoder.encoder(img_tensor)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 3.71 GiB of which 895.19 MiB is free. Including non-PyTorch memory, this process has 2.82 GiB memory in use. Of the allocated memory 2.63 GiB is allocated by PyTorch, and 88.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
torch.cuda.empty_cache()